In [2]:
%pylab
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.cross_decomposition import PLSRegression
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import zscore
import mplcursors


DeSeqOutputAllConds = 'DeSeqOutputAllConds.tsv'
si2_si4_RNA_seq = 'si2-si4_RNA-seq-pipeline-output-normalized.tsv'
le = LabelEncoder()
oh_enc = OneHotEncoder(handle_unknown='ignore')
plsr = PLSRegression(n_components=2)

#https://stackoverflow.com/questions/11346283/renaming-columns-in-pandas
def combineTreatmentColumns(df, exceptions):
    df.rename(columns=lambda x: x[3:] if x not in exceptions else x, inplace=True)
    return df

def changeTreatmentNameInRows(df, columnName='sampleID', exceptions={}):
    df[columnName] = df[columnName].apply(lambda x: x[3:] if x not in exceptions else x)
    return df


def getTSVFile(filepath):
    data = pd.read_csv (filepath, sep = '\t')
    return data

#not used or necessary
def integerEncode(dataframe, columnNametoEncode, newEncodedColumnName):
    dataframe[columnNametoEncode] = le.fit_transform(df[newEncodedColumnName])
    return dataframe
  
#https://stackoverflow.com/questions/37292872/how-can-i-one-hot-encode-in-python  
def oneHotEncode(df, columnsToEncode):
    
    for column in columnsToEncode:
        # Get one hot encoding of column and drop 'column'
        one_hot = pd.get_dummies(df[column])
        # Drop column  as it is now encoded
        df = df.drop(column, axis = 1)
        # Join the encoded df
        df = df.join(one_hot)
    
    return df


def zScoreData(df, columnsToZScore):
    for column in columnsToZScore:
        df[column] = zscore(df[column])
        
    return df

def fixBadChromosomeLabeling(df, column='chrom'): 
    replace = list(range(1, 23)) + ['X', 'Y']
    
    for i in replace:
        df[column].replace(str(i), 'chr' + str(i), inplace=True)
        
    return df


Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [3]:
df = getTSVFile(si2_si4_RNA_seq)
df = df.loc[df['counts'] > 0]#Filter out rows where count is zero
df = fixBadChromosomeLabeling(df)

#Set up copies to play around with lader
df_original = df.copy()

#Remove unnecessary columns
df = df[['sampleID', 'tpm', 'gene_name']]

#display(df.sampleID.unique())

df = oneHotEncode(df, ['sampleID'])
df = zScoreData(df, ['tpm'])


# Leon if you want to get top n%
# Probably simpler - https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.nlargest.html
# Sort, .head(...) - https://stackoverflow.com/questions/53405458/selecting-the-top-50-percentage-names-from-the-columns-of-a-pandas-dataframe



In [4]:
#Just a cell for testing the "changeTreatmentNameInRows" function which 
#removes the number '31-' from '31-RA-high' and groups them to see new values

df_test = df_original.copy()
df_test = df_test.loc[df_test['counts'] > 0]

#changeTreatmentNameInRows(df_test)

#Homework: Get top 10 results, for gene_name, sampleID, and both and PLOT IT
#df_test.groupby(['sampleID' ]).mean().sort_values(by=['tpm'], ascending=False)



In [5]:
# Retinoic Acid (50, 200 and 400 nM)
# TGF-β (1.25, 5, and 10 ng/mL)
# EtOH - Ethanol control

#Replace rows w/ 'TGFb-and-RA-high' with the numerical values of 'TGFb' and 'Retinoic Acid'
def replaceTreatmentsInDF(df):
    for treatment in sampleIDs_transform.keys():
        sit = sampleIDs_transform[treatment]
        df.loc[df.sampleID == treatment, ['TGFb', 'Retinoic Acid']] = sit['TGFb'], sit['retinoicAcid']
    return df

#Transforms columns with these keys to columns with these values
sampleIDs_transform = {
  'RA-high': {'TGFb': 0, 'retinoicAcid': 400, 'EtOH': 0}, 
  'EtOH-nlDensity': {'TGFb': 0, 'retinoicAcid': 0, 'EtOH': 0},
  'RA-med': {'TGFb': 0, 'retinoicAcid': 200, 'EtOH': 0}, 
  'EtOH-halfDensity': {'TGFb': 0, 'retinoicAcid': 0, 'EtOH': 0}, 
  'RA-low': {'TGFb': 0, 'retinoicAcid': 50, 'EtOH': 0},
  'EtOH-highDensity': {'TGFb': 0, 'retinoicAcid': 0, 'EtOH': 0}, 
  'TGFb-and-RA-low': {'TGFb': 1.25, 'retinoicAcid': 50, 'EtOH': 0}, 
  'TGFb-and-RA-high': {'TGFb': 10, 'retinoicAcid': 400, 'EtOH': 0},
  'TGFb-and-RA-med': {'TGFb': 5, 'retinoicAcid': 200, 'EtOH': 0}, 
  'TGFb-high': {'TGFb': 10, 'retinoicAcid': 0, 'EtOH': 0}, 
  'TGFb-low': {'TGFb': 1.25, 'retinoicAcid': 0, 'EtOH': 0}, 
  'TGFb-med': {'TGFb': 5, 'retinoicAcid': 0, 'EtOH': 0}
}

In [6]:

    
#Simply takes the X and Y loadings and plots them with their legend
def printPLSRPlot(x_loadings_, y_loadings_, loadings, hoverName=True, printY=True):
    if printY:
        plt.scatter(y_loadings_[:, 0], y_loadings_[:, 1], color='blue', label='tpm')
    
    for i in range(x_loadings_.shape[0]):
        plt.scatter(x_loadings_[i,0], x_loadings_[i, 1], label=loadings[i])
        plt.annotate(loadings[i], (x_loadings_[i,0], x_loadings_[i, 1]))
    
    
    
    #https://stackoverflow.com/questions/7908636/possible-to-make-labels-appear-when-hovering-over-a-point-in-matplotlib
    mplcursors.cursor(hover=hoverName)
    
    
    plt.legend(bbox_to_anchor=(1.53, 1))
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.show()
  
#Creates a new datatable by grouping by two columns then returning a new 
#datatable where the rows were the first groupby element and the columns were the 2nd groupby
def getPLSR_DFs(df_plsr, Y_label='tpm'):   
    df_plsr = df_plsr.pivot_table(index=['gene_name'], columns='sampleID', values='counts', fill_value=0)

    return df_plsr, df_plsr[Y_label]


#Uses "sampleIDs_transform" to put in the correct treatments in their 
#appropriate columns
def alterTreatmentsInDF(new_df, df, newColumns=['TGFb', 'retinoicAcid']):
    
    for i in range(len(new_df.index)):
        
        (gene_name, treatment) = df.iloc[i].name
        new_df.iat[i, 0] = sampleIDs_transform[treatment]['TGFb']
        new_df.iat[i, 1]= sampleIDs_transform[treatment]['retinoicAcid']
        #new_df.iat[i, 2]= sampleIDs_transform[treatment]['EtOH']
        
    return new_df

#Takes a multiindex dataframe, creates a new dataframe, and inputs the correct 
#treatments into it
def createPLSRMatrix(df, newColumns=['TGFb', 'retinoicAcid']):
    #Create a new dataframe    
    new_indexes = [gene for (gene, treatment) in df.index]
    new_df = pd.DataFrame(0, index=new_indexes, columns=newColumns)
            
    return alterTreatmentsInDF(new_df, df)




In [7]:
#In this section Im just fitting the new dataframe with columns 
#"TGF-b" and "Retinoic Acid" to results
df_plsr = df_original.copy()
df_plsr = df_plsr[['sampleID', 'tpm', 'counts', 'gene_name']]
changeTreatmentNameInRows(df_plsr, columnName='sampleID')

#Leon ur having trouble deciding what goes into the values for X matrix and what is left for Y
# new_df, Y = getPLSR_DFs(df_plsr)
# display(Y)


#Original
df_plsr = df_plsr.groupby(['gene_name', 'sampleID']).mean().sort_values(by=['tpm'], ascending=False)
Y = df_plsr['tpm']

display(df_plsr)


better_df = createPLSRMatrix(df_plsr)

loadings = [col for col in df_plsr.columns if col != 'tpm'] #Remove 'tpm' to fit for PLSR
plsr.fit(better_df, Y)
print('PLSR Variance captured: ', plsr.score(better_df, Y))
printPLSRPlot(plsr.x_loadings_, plsr.y_loadings_, better_df.columns)

tpm        counts
gene_name sampleID                                    
MT-CO2    TGFb-and-RA-high  31757.766300  58321.666667
          EtOH-highDensity  31195.055071  57061.333333
          TGFb-high         31161.369708  61274.666667
          RA-high           29523.226736  55900.666667
          EtOH-halfDensity  29201.088032  54290.666667
...                                  ...           ...
FLRT2     EtOH-nlDensity        0.010230      1.000000
          TGFb-low              0.009743      1.000000
TSIX      TGFb-low              0.009234      1.000000
FLRT2     TGFb-and-RA-high      0.009183      1.000000
CACNA1A   EtOH-halfDensity      0.008586      1.000000

[220367 rows x 2 columns]

PLSR Variance captured:  2.386067772253142e-07


In [8]:
def getDF_PLSR_Pivoted(values='tpm', index=['TGFb', 'Retinoic Acid'], columns='gene_name', reset_index=True):
    df_plsr = df_original.copy()
    #Only need these columns so remove everything else
    df_plsr = df_plsr[['sampleID', 'tpm', 'counts', 'gene_name']]

    changeTreatmentNameInRows(df_plsr, columnName='sampleID')

    #Zero out NaN values
    df_plsr[['TGFb', 'Retinoic Acid']] = 0
    replaceTreatmentsInDF(df_plsr)

    #Group new datable by treatments and make the columns have the TPM values
    df_plsr = df_plsr.pivot_table(index=index, columns=columns, values=values, fill_value=0)
    
    #Undo the pivot table to make dataframe all columns 
    if reset_index:
        df_plsr.reset_index(inplace=True)

    return df_plsr

In [9]:

plsr = PLSRegression(n_components=2)


df_plsr = getDF_PLSR_Pivoted(reset_index=True)


#Make a dataframe of just genes
treatments = ['TGFb', 'Retinoic Acid']
genes = df_plsr[df_plsr.columns.difference(treatments)]

display(df_plsr)
display(genes)



gene_name,TGFb,Retinoic Acid,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,0.00,0,0.377997,0.035485,0.087716,0.263830,0.350411,12.057979,0.319075,53.020671,...,37.376160,182.345536,3.376170,9.657581,36.781075,16.005638,18.488987,72.885622,33.661747,22.205224
1,0.00,50,0.345151,0.034716,0.127103,0.137183,0.000000,13.425112,0.000000,47.729696,...,37.457048,150.343398,2.518407,9.277073,33.313392,16.573176,20.549266,107.455344,33.147009,23.398570
2,0.00,200,0.300917,0.000000,0.162882,0.082244,0.345012,13.311664,0.000000,47.800100,...,30.569839,130.592939,2.773964,7.611645,33.531139,15.295295,20.328332,118.413674,32.668651,22.805652
3,0.00,400,0.203307,0.000000,0.211807,0.069556,0.335723,13.785032,0.000000,46.066403,...,32.521534,123.678421,2.668080,8.206404,32.979449,15.033746,20.089239,121.144959,30.442728,23.403616
4,1.25,0,0.256296,0.000000,0.080188,0.125608,0.000000,11.649623,0.000000,45.712356,...,27.967913,126.723619,3.286000,11.075037,37.483739,14.895510,17.376096,149.058857,32.772074,23.163784
5,1.25,50,0.351086,0.000000,0.054925,0.075671,0.385303,8.973253,0.000000,41.863051,...,27.021822,74.655933,3.736386,10.136228,37.666414,13.484046,21.128904,194.584197,32.281384,23.780587
6,5.00,0,0.230624,0.035837,0.053812,0.110743,0.336408,9.336744,0.193978,43.406801,...,32.606337,110.171030,3.162182,11.979568,35.755054,14.319992,19.077618,153.719588,32.896717,23.078828
7,5.00,200,0.362810,0.000000,0.054184,0.050074,0.676265,9.562037,0.000000,42.264973,...,25.416376,66.817892,3.253249,10.199674,36.682645,13.111032,21.180285,207.092959,31.524216,24.633575
8,10.00,0,0.350926,0.032057,0.092482,0.130880,0.358890,10.128333,0.000000,43.736571,...,28.349972,100.414759,3.536064,12.396366,36.297142,13.755184,18.309939,161.319258,32.275104,22.633430
9,10.00,400,0.419643,0.046901,0.000000,0.098149,0.376772,10.329859,0.000000,40.635957,...,22.338756,57.523884,3.284246,9.952191,36.076187,13.478254,22.231341,204.279198,29.913855,25.225166


gene_name,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,0.377997,0.035485,0.087716,0.263830,0.350411,12.057979,0.319075,53.020671,20.904988,0.338778,...,37.376160,182.345536,3.376170,9.657581,36.781075,16.005638,18.488987,72.885622,33.661747,22.205224
1,0.345151,0.034716,0.127103,0.137183,0.000000,13.425112,0.000000,47.729696,21.923315,0.227967,...,37.457048,150.343398,2.518407,9.277073,33.313392,16.573176,20.549266,107.455344,33.147009,23.398570
2,0.300917,0.000000,0.162882,0.082244,0.345012,13.311664,0.000000,47.800100,22.605397,0.000000,...,30.569839,130.592939,2.773964,7.611645,33.531139,15.295295,20.328332,118.413674,32.668651,22.805652
3,0.203307,0.000000,0.211807,0.069556,0.335723,13.785032,0.000000,46.066403,20.853942,0.000000,...,32.521534,123.678421,2.668080,8.206404,32.979449,15.033746,20.089239,121.144959,30.442728,23.403616
4,0.256296,0.000000,0.080188,0.125608,0.000000,11.649623,0.000000,45.712356,21.377179,0.000000,...,27.967913,126.723619,3.286000,11.075037,37.483739,14.895510,17.376096,149.058857,32.772074,23.163784
5,0.351086,0.000000,0.054925,0.075671,0.385303,8.973253,0.000000,41.863051,22.616414,0.242189,...,27.021822,74.655933,3.736386,10.136228,37.666414,13.484046,21.128904,194.584197,32.281384,23.780587
6,0.230624,0.035837,0.053812,0.110743,0.336408,9.336744,0.193978,43.406801,20.842553,0.212957,...,32.606337,110.171030,3.162182,11.979568,35.755054,14.319992,19.077618,153.719588,32.896717,23.078828
7,0.362810,0.000000,0.054184,0.050074,0.676265,9.562037,0.000000,42.264973,22.752890,0.207380,...,25.416376,66.817892,3.253249,10.199674,36.682645,13.111032,21.180285,207.092959,31.524216,24.633575
8,0.350926,0.032057,0.092482,0.130880,0.358890,10.128333,0.000000,43.736571,20.615554,0.184561,...,28.349972,100.414759,3.536064,12.396366,36.297142,13.755184,18.309939,161.319258,32.275104,22.633430
9,0.419643,0.046901,0.000000,0.098149,0.376772,10.329859,0.000000,40.635957,21.016866,0.231078,...,22.338756,57.523884,3.284246,9.952191,36.076187,13.478254,22.231341,204.279198,29.913855,25.225166


In [24]:
##########################################################
# Plot scores of sample treatments 
# where counts=X and tpm=Y
##########################################################
df_plsr = df_original.copy()
df_plsr = df_plsr[['sampleID', 'tpm', 'counts', 'gene_name']]

changeTreatmentNameInRows(df_plsr, columnName='sampleID')
df_plsr_X = df_plsr.pivot_table(index='sampleID', columns='gene_name', values='counts', fill_value=0)
df_plsr_Y = df_plsr.pivot_table(index='sampleID', columns='gene_name', values='tpm', fill_value=0)

# display(df_plsr_X)
# display(df_plsr_Y)

plsr.fit(df_plsr_X, df_plsr_Y)
print('PLSR Variance captured: ', plsr.score(df_plsr_X, df_plsr_Y))
printPLSRPlot(plsr.x_scores_, plsr.y_scores_, df_plsr_X.index, printY=False)



PLSR Variance captured:  0.5301826840341864


In [23]:
#################################################################
# This cell captures the variance of top percentage of genes
# when undergoing PLSR
#################################################################



treatments, variance_capture, num_genes = ['TGFb', 'Retinoic Acid'], [], []
df_plsr = getDF_PLSR_Pivoted() #df_plsr.pivot_table(index=['TGFb', 'Retinoic Acid'], columns='gene_name', values='tpm', fill_value=0)

display(df_plsr)

for i in [0, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 0.999, 0.9999]:
    percentile = np.quantile(genes.sum(axis = 0), i)
    screened_genes = genes[genes.columns[genes.sum() > percentile]]
    
    
    plsr.fit(df_plsr[treatments], screened_genes)
    variance_score = plsr.score(df_plsr[treatments], screened_genes)
    variance_capture.append(variance_score)
    num_genes.append(screened_genes.shape[1])
    print('PLSR Variance captured: ', variance_score)


#Only percentages between 0 and 1
plt.ylim(0, 1)
#Make x label to show how many # of genes vs captured variance
labels = ['%d' % i for i in num_genes]

#Just to make a readable x axis plot
genes_count = np.linspace(0, 9, len(num_genes))

plt.plot(genes_count, variance_capture)
plt.xticks(range(len(labels)), labels, rotation='vertical')

plt.title('')
plt.tight_layout()
plt.xlabel('Top N Genes')
plt.ylabel('Percent of Variance Captured')
plt.show()



gene_name,TGFb,Retinoic Acid,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,0.00,0,0.377997,0.035485,0.087716,0.263830,0.350411,12.057979,0.319075,53.020671,...,37.376160,182.345536,3.376170,9.657581,36.781075,16.005638,18.488987,72.885622,33.661747,22.205224
1,0.00,50,0.345151,0.034716,0.127103,0.137183,0.000000,13.425112,0.000000,47.729696,...,37.457048,150.343398,2.518407,9.277073,33.313392,16.573176,20.549266,107.455344,33.147009,23.398570
2,0.00,200,0.300917,0.000000,0.162882,0.082244,0.345012,13.311664,0.000000,47.800100,...,30.569839,130.592939,2.773964,7.611645,33.531139,15.295295,20.328332,118.413674,32.668651,22.805652
3,0.00,400,0.203307,0.000000,0.211807,0.069556,0.335723,13.785032,0.000000,46.066403,...,32.521534,123.678421,2.668080,8.206404,32.979449,15.033746,20.089239,121.144959,30.442728,23.403616
4,1.25,0,0.256296,0.000000,0.080188,0.125608,0.000000,11.649623,0.000000,45.712356,...,27.967913,126.723619,3.286000,11.075037,37.483739,14.895510,17.376096,149.058857,32.772074,23.163784
5,1.25,50,0.351086,0.000000,0.054925,0.075671,0.385303,8.973253,0.000000,41.863051,...,27.021822,74.655933,3.736386,10.136228,37.666414,13.484046,21.128904,194.584197,32.281384,23.780587
6,5.00,0,0.230624,0.035837,0.053812,0.110743,0.336408,9.336744,0.193978,43.406801,...,32.606337,110.171030,3.162182,11.979568,35.755054,14.319992,19.077618,153.719588,32.896717,23.078828
7,5.00,200,0.362810,0.000000,0.054184,0.050074,0.676265,9.562037,0.000000,42.264973,...,25.416376,66.817892,3.253249,10.199674,36.682645,13.111032,21.180285,207.092959,31.524216,24.633575
8,10.00,0,0.350926,0.032057,0.092482,0.130880,0.358890,10.128333,0.000000,43.736571,...,28.349972,100.414759,3.536064,12.396366,36.297142,13.755184,18.309939,161.319258,32.275104,22.633430
9,10.00,400,0.419643,0.046901,0.000000,0.098149,0.376772,10.329859,0.000000,40.635957,...,22.338756,57.523884,3.284246,9.952191,36.076187,13.478254,22.231341,204.279198,29.913855,25.225166


PLSR Variance captured:  0.4004045357126894
PLSR Variance captured:  0.4522931336024578
PLSR Variance captured:  0.49686683462375353
PLSR Variance captured:  0.5126218770309794
PLSR Variance captured:  0.527031817796695
PLSR Variance captured:  0.5331246230376915
PLSR Variance captured:  0.5079130615548616
PLSR Variance captured:  0.3615562412977845
PLSR Variance captured:  0.32411942594593995


<ipython-input-23-6570cb663ed8>:36: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations. 
  plt.tight_layout()


In [24]:
#################################################################
# This fits the treatments(numerical samples) to the genes' tpm
# values and then predicts some sample inputs
#################################################################

treatments = ['TGFb', 'Retinoic Acid']
df_plsr = getDF_PLSR_Pivoted() #df_plsr.pivot_table(index=['TGFb', 'Retinoic Acid'], columns='gene_name', values='tpm', fill_value=0)

display(df_plsr)

#Fit treatments and predict
plsr.fit(df_plsr[treatments], genes)
print('PLSR Variance captured: ', plsr.score(df_plsr[treatments], genes))
#printPLSRPlot(plsr.x_loadings_, plsr.y_loadings_, treatments)

#Predict values of new treatment
Y_pred = plsr.predict([[10, 400], [14, 300]])

treatments = ['TGFb', 'Retinoic Acid']


df_pred = pd.DataFrame(Y_pred, columns=genes.columns)
display(pd.DataFrame(np.array([[10, 400], [14, 300]]), columns=treatments).join(df_pred))



gene_name,TGFb,Retinoic Acid,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,0.00,0,0.377997,0.035485,0.087716,0.263830,0.350411,12.057979,0.319075,53.020671,...,37.376160,182.345536,3.376170,9.657581,36.781075,16.005638,18.488987,72.885622,33.661747,22.205224
1,0.00,50,0.345151,0.034716,0.127103,0.137183,0.000000,13.425112,0.000000,47.729696,...,37.457048,150.343398,2.518407,9.277073,33.313392,16.573176,20.549266,107.455344,33.147009,23.398570
2,0.00,200,0.300917,0.000000,0.162882,0.082244,0.345012,13.311664,0.000000,47.800100,...,30.569839,130.592939,2.773964,7.611645,33.531139,15.295295,20.328332,118.413674,32.668651,22.805652
3,0.00,400,0.203307,0.000000,0.211807,0.069556,0.335723,13.785032,0.000000,46.066403,...,32.521534,123.678421,2.668080,8.206404,32.979449,15.033746,20.089239,121.144959,30.442728,23.403616
4,1.25,0,0.256296,0.000000,0.080188,0.125608,0.000000,11.649623,0.000000,45.712356,...,27.967913,126.723619,3.286000,11.075037,37.483739,14.895510,17.376096,149.058857,32.772074,23.163784
5,1.25,50,0.351086,0.000000,0.054925,0.075671,0.385303,8.973253,0.000000,41.863051,...,27.021822,74.655933,3.736386,10.136228,37.666414,13.484046,21.128904,194.584197,32.281384,23.780587
6,5.00,0,0.230624,0.035837,0.053812,0.110743,0.336408,9.336744,0.193978,43.406801,...,32.606337,110.171030,3.162182,11.979568,35.755054,14.319992,19.077618,153.719588,32.896717,23.078828
7,5.00,200,0.362810,0.000000,0.054184,0.050074,0.676265,9.562037,0.000000,42.264973,...,25.416376,66.817892,3.253249,10.199674,36.682645,13.111032,21.180285,207.092959,31.524216,24.633575
8,10.00,0,0.350926,0.032057,0.092482,0.130880,0.358890,10.128333,0.000000,43.736571,...,28.349972,100.414759,3.536064,12.396366,36.297142,13.755184,18.309939,161.319258,32.275104,22.633430
9,10.00,400,0.419643,0.046901,0.000000,0.098149,0.376772,10.329859,0.000000,40.635957,...,22.338756,57.523884,3.284246,9.952191,36.076187,13.478254,22.231341,204.279198,29.913855,25.225166


PLSR Variance captured:  0.40039272982335644


,TGFb,Retinoic Acid,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,10,400,0.362298,0.029668,0.054499,0.054887,0.518592,10.323523,-0.043468,40.068333,...,22.720921,52.020810,3.206369,10.153712,35.357228,13.032290,21.708999,207.673088,29.807358,24.841513
1,14,300,0.393041,0.042628,0.004124,0.067310,0.545893,8.713564,-0.029093,38.268409,...,20.779741,35.966035,3.508953,11.859800,36.595315,12.351237,21.193319,230.377725,30.033920,24.827794


In [ ]:
#################################################################
# Import Support Vector Regression 
#################################################################
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

#https://www.machinecurve.com/index.php/2020/11/17/how-to-perform-multioutput-regression-with-svms-in-python/

regr = make_pipeline(StandardScaler(), SVR(kernel='poly', C=1.0, epsilon=0.2))
regr.fit(df_plsr[treatments], genes)
#Y_pred = regr.predict([[10, 400], [14, 300]])

display(Y_pred)

In [18]:
#################################################################
# Support Vector Regression MultiOutput
#
# !CANNOT WORK: SVR cannot handle multiple outputs!
#################################################################
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, mean_absolute_error, explained_variance_score


treatments = ['TGFb', 'Retinoic Acid']
df_plsr = getDF_PLSR_Pivoted() #df_plsr.pivot_table(index=['TGFb', 'Retinoic Acid'], columns='gene_name', values='tpm', fill_value=0)

display(df_plsr)
gene_names = ['A1BG']
genes = df_plsr[df_plsr.columns.difference(treatments)]

#Fit treatments and predict
svr = SVR(epsilon=0.2)
svr.fit(df_plsr[treatments], genes[gene_names])



# X, y = make_regression(n_samples=20, n_features=2000, n_targets=250, random_state=33)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=33)

#only use single gene

# Create the SVR regressor
# svr = SVR(epsilon=0.2)
# svr.fit(X_train, y_train)


# Train the regressor
#mor = mor.fit(X_train, y_train)

# Generate predictions for testing data
#y_pred = mor.predict(X_test)

print(svr.predict([[10, 400]]))
print(svr.predict([[0, 0]]))

# Evaluate the regressor
mse_one = svr.score(df_plsr[treatments], genes[gene_names])
#mse_two = r2_score(y_test[:,1], y_pred[:,1])


print(f'Score for svr: {mse_one}')
# mae_one = mean_absolute_error(y_test[:,0], y_pred[:,0])
# mae_two = mean_absolute_error(y_test[:,1], y_pred[:,1])
# print(f'MAE for first regressor: {mae_one} - second regressor: {mae_two}')


gene_name,TGFb,Retinoic Acid,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,0.00,0,0.377997,0.035485,0.087716,0.263830,0.350411,12.057979,0.319075,53.020671,...,37.376160,182.345536,3.376170,9.657581,36.781075,16.005638,18.488987,72.885622,33.661747,22.205224
1,0.00,50,0.345151,0.034716,0.127103,0.137183,0.000000,13.425112,0.000000,47.729696,...,37.457048,150.343398,2.518407,9.277073,33.313392,16.573176,20.549266,107.455344,33.147009,23.398570
2,0.00,200,0.300917,0.000000,0.162882,0.082244,0.345012,13.311664,0.000000,47.800100,...,30.569839,130.592939,2.773964,7.611645,33.531139,15.295295,20.328332,118.413674,32.668651,22.805652
3,0.00,400,0.203307,0.000000,0.211807,0.069556,0.335723,13.785032,0.000000,46.066403,...,32.521534,123.678421,2.668080,8.206404,32.979449,15.033746,20.089239,121.144959,30.442728,23.403616
4,1.25,0,0.256296,0.000000,0.080188,0.125608,0.000000,11.649623,0.000000,45.712356,...,27.967913,126.723619,3.286000,11.075037,37.483739,14.895510,17.376096,149.058857,32.772074,23.163784
5,1.25,50,0.351086,0.000000,0.054925,0.075671,0.385303,8.973253,0.000000,41.863051,...,27.021822,74.655933,3.736386,10.136228,37.666414,13.484046,21.128904,194.584197,32.281384,23.780587
6,5.00,0,0.230624,0.035837,0.053812,0.110743,0.336408,9.336744,0.193978,43.406801,...,32.606337,110.171030,3.162182,11.979568,35.755054,14.319992,19.077618,153.719588,32.896717,23.078828
7,5.00,200,0.362810,0.000000,0.054184,0.050074,0.676265,9.562037,0.000000,42.264973,...,25.416376,66.817892,3.253249,10.199674,36.682645,13.111032,21.180285,207.092959,31.524216,24.633575
8,10.00,0,0.350926,0.032057,0.092482,0.130880,0.358890,10.128333,0.000000,43.736571,...,28.349972,100.414759,3.536064,12.396366,36.297142,13.755184,18.309939,161.319258,32.275104,22.633430
9,10.00,400,0.419643,0.046901,0.000000,0.098149,0.376772,10.329859,0.000000,40.635957,...,22.338756,57.523884,3.284246,9.952191,36.076187,13.478254,22.231341,204.279198,29.913855,25.225166


[0.31147474]
[0.31147474]
Score for svr: -0.01614692793854


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [ ]:
#################################################################
# Support Vector Regression MultiOutput
#################################################################
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, mean_absolute_error, explained_variance_score

X, y = make_regression(n_samples=20, n_features=2000, n_targets=250, random_state=33)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=33)

# Create the SVR regressor
svr = SVR(epsilon=0.2)
mor = MultiOutputRegressor(svr)


# Train the regressor
mor = mor.fit(X_train, y_train)

# Generate predictions for testing data
y_pred = mor.predict(X_test)

# Evaluate the regressor
mse_one = r2_score(y_test, y_pred)
#mse_two = r2_score(y_test[:,1], y_pred[:,1])


print(f'MSE for first regressor: {mse_one} - second regressor: {mse_two}')
# mae_one = mean_absolute_error(y_test[:,0], y_pred[:,0])
# mae_two = mean_absolute_error(y_test[:,1], y_pred[:,1])
# print(f'MAE for first regressor: {mae_one} - second regressor: {mae_two}')


In [20]:
#################################################################
# A cell to import relevant tf classes
#################################################################

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing



In [23]:
#################################################################
# Neural Network Disaster
#################################################################


#################################################################
# Trying for multioutput neural network regression
# https://towardsdatascience.com/multi-output-model-with-tensorflow-keras-functional-api-875dd89aa7c6
#################################################################
# input_layers = tf.keras.Input(shape=(2, ))
# previous_layer = input_layers

# layer = tf.keras.layers.Dense(units=3, activation='relu')(previous_layer)
# op = tf.keras.layers.Dense(units=1)(layer)
# output_layers= [op]


# for name in range(2):
#     layer = tf.keras.layers.Dense(units=32, activation='relu')(previous_layer)
#     output = tf.keras.layers.Dense(units=1)(layer)
#     output_layers.append(output)
#     previous_layer = layer
# model = tf.keras.Model(input_layers, output_layers)

#################################################################
# Can we predict using a 600,000+ x 20,000 matrix (observations vs genes)
#
#
# No ; - ( 
# https://stackoverflow.com/questions/53703431/support-vector-regression-multiple-outputs
#################################################################
df_plsr = df_original.copy()
df_plsr = df_plsr[['sampleID', 'tpm', 'counts', 'gene_name']]

# changeTreatmentNameInRows(df_plsr, columnName='sampleID')
# df_plsr_X = df_plsr.pivot_table(index='sampleID', columns='gene_name', values='counts', fill_value=0)
# df_plsr_Y = df_plsr.pivot_table(index='sampleID', columns='gene_name', values='tpm', fill_value=0)

display(df_plsr.pivot_table(columns='gene_name', fill_value=0))

df_plsr = getDF_PLSR_Pivoted()
treatments = ['TGFb', 'Retinoic Acid']
gene_names = ['AADAC']
genes = df_plsr[df_plsr.columns.difference(treatments)]

model = tf.keras.Sequential([
    layers.Dense(units='2'), #For each input column
    layers.Dense(units='32'),
    layers.Dense(units='32'),
    layers.Dense(units='1')
])

model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.01),
    loss='mean_absolute_error',
    metrics=[tf.keras.metrics.Accuracy()]
)

model.fit(
    df_plsr[treatments], genes[gene_names],
    epochs=100,
    # suppress logging
    #verbose=0,
    # Calculate validation results on 20% of the training data
    #validation_split = 0.2
)

print(model.predict([[10, 400]]))



gene_name,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
counts,3.638889,1.000000,1.900000,3.03125,1.090909,92.891892,1.000000,533.783784,950.216216,1.000000,...,421.162162,824.027027,36.162162,162.270270,728.081081,192.459459,442.675676,1294.729730,1420.756757,576.972973
tpm,0.329563,0.036747,0.103047,0.15150,0.388352,11.407696,0.235677,46.698768,21.428708,0.232105,...,31.527308,125.573540,3.200471,9.975091,35.869358,14.862029,19.613593,134.596225,32.442775,23.200591


Epoch 1/100
1/1 [==============================] - 0s 409ms/step - loss: 23.1595 - accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 3ms/step - loss: 27.1537 - accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 3ms/step - loss: 23.0595 - accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 3ms/step - loss: 4.5999 - accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 2ms/step - loss: 18.9718 - accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 4ms/step - loss: 26.3096 - accuracy: 0.0000e+00
Epoch 7/100
1/1 [==============================] - 0s 3ms/step - loss: 24.1147 - accuracy: 0.0000e+00
Epoch 8/100
1/1 [==============================] - 0s 3ms/step - loss: 16.2564 - accuracy: 0.0000e+00
Epoch 9/100
1/1 [==============================] - 0s 6ms/step - loss: 5.0596 - accuracy: 0.0000e+00
Epoch 10/100
1/1 [==============================] - 0s 3ms/step - loss: 8.3025 - a

1/1 [==============================] - 0s 19ms/step - loss: 0.4945 - accuracy: 0.0000e+00
Epoch 81/100
1/1 [==============================] - 0s 5ms/step - loss: 0.0885 - accuracy: 0.0000e+00
Epoch 82/100
1/1 [==============================] - 0s 7ms/step - loss: 0.4196 - accuracy: 0.0000e+00
Epoch 83/100
1/1 [==============================] - 0s 3ms/step - loss: 0.5289 - accuracy: 0.0000e+00
Epoch 84/100
1/1 [==============================] - 0s 7ms/step - loss: 0.3517 - accuracy: 0.0000e+00
Epoch 85/100
1/1 [==============================] - 0s 3ms/step - loss: 0.1520 - accuracy: 0.0000e+00
Epoch 86/100
1/1 [==============================] - 0s 5ms/step - loss: 0.2888 - accuracy: 0.0000e+00
Epoch 87/100
1/1 [==============================] - 0s 3ms/step - loss: 0.1419 - accuracy: 0.0000e+00
Epoch 88/100
1/1 [==============================] - 0s 11ms/step - loss: 0.3087 - accuracy: 0.0000e+00
Epoch 89/100
1/1 [==============================] - 0s 4ms/step - loss: 0.3800 - accuracy: 0.

In [ ]:
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D

pca = PCA(n_components=3)

#Guys if you increase the number of PCs you get a lot more variance capture


def printPCA_2D(new_df, title):
    pca.fit(new_df)

    print('''
    Variance for PCs = %s 
    Sum of variances = %.03f  
    pca.shape = %s where rows=loadings=%d, columns=PCs=%d''' 
          % (pca.explained_variance_ratio_, np.sum(pca.explained_variance_ratio_), 
             pca.components_.shape, pca.components_.shape[1], pca.components_.shape[0]))
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.title(title)

    for i in range(pca.components_.shape[1]):
        plt.scatter(pca.components_[0, i], pca.components_[1, i], label=new_df.columns[i])

    #https://stackoverflow.com/questions/7908636/possible-to-make-labels-appear-when-hovering-over-a-point-in-matplotlib
    mplcursors.cursor(hover=True)

    plt.legend(bbox_to_anchor=(1.35, 1)) 
    plt.show()

    
def printPCA_3D(new_df, title):
    pca.fit(new_df)

    print('''
    Variance for PCs = %s 
    Sum of variances = %.03f  
    pca.shape = %s where rows=loadings=%d, columns=PCs=%d''' 
          % (pca.explained_variance_ratio_, np.sum(pca.explained_variance_ratio_), 
             pca.components_.shape, pca.components_.shape[1], pca.components_.shape[0]))
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.title(title)

    for i in range(pca.components_.shape[1]):
        ax.scatter(xs=pca.components_[0, i], ys=pca.components_[1, i], 
                       zs=pca.components_[2, i], label=new_df.columns[i])


    plt.legend(bbox_to_anchor=(1.6, 1)) 
    plt.show()


def getPCA_DFs(df_pca, groupby=['sampleID', 'gene_name'], keepColumns=['sampleID', 'tpm', 'gene_name'], topN=5000):
    
    df_pca = df_pca[keepColumns]

    #Group data by two columns, which will later be a regular row and column
    gb = df_pca.groupby(groupby).mean().sort_values(by=['tpm'], ascending=False)[:topN]
    
    
    display(gb)
    #Extract rows and columns of top N results by indices
    rows = set([groupby[0] for groupby in gb.index])
    columns = set([groupby[1] for groupby in gb.index])

    #Create a new dataframe
    new_df = pd.DataFrame(0, index=rows, columns=columns)
    new_df_log = new_df.copy()

    #Iterate
    for (row, col) in gb.index:
        tpv_val = gb.loc[row, col].tpm
        new_df.loc[row, col] = tpv_val
        new_df_log.loc[row, col] = np.log(tpv_val)
        
    return new_df, new_df_log


#From Meyer treatments as rows, genes as columns

df_pca = df_original.copy()

groupby=['sampleID', 'gene_name'] #Should only be two things! groupby[0] = row, groupby[1] = column
changeTreatmentNameInRows(df_pca, columnName='sampleID')

new_df, new_df_log = getPCA_DFs(df_pca, groupby=groupby, topN=5000)

#display(new_df)
print()
printPCA_2D(new_df_log, title='Loadings Plot for Log-Normal PCA' )

print()
printPCA_2D(new_df, title='Loadings Plot for Normal PCA' )



In [99]:
#################################################################
# Decision Tree
#################################################################
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

treatments = ['TGFb', 'Retinoic Acid']
df_dt = getDF_PLSR_Pivoted()
index = 9

print(f'Lets take out row {index}')
display(df_dt.iloc[[index]])
print('Now this is what our new table looks like')
df_dt_minus_row = df_dt.drop(df_dt.index[[index]])
display(df_dt_minus_row)

genes = df_dt_minus_row[df_dt_minus_row.columns.difference(treatments)]

regressor = DecisionTreeRegressor(random_state=0)

#"cv" cannot be greater than 4 cus I only have 10 samples
# Uses r2 scoring
scores = cross_val_score(regressor, df_dt_minus_row[treatments], genes, cv=4, scoring='explained_variance')

print(f'Cross validation scores: {scores}')
sample_treatment = [[10, 400]]

regressor.fit(df_dt_minus_row[treatments], genes)

print(f'Decision Tree Regressor score is {regressor.score(df_dt_minus_row[treatments], genes)} \n\n\n')

print('Y true')
display(df_dt.iloc[[index]])

print('Y predict')
Y_pred = pd.DataFrame(regressor.predict(sample_treatment), columns=genes.columns)
display(pd.DataFrame(np.array(sample_treatment), columns=treatments).join(Y_pred))




Lets take out row 9


gene_name,TGFb,Retinoic Acid,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
9,10.0,400,0.419643,0.046901,0.0,0.098149,0.376772,10.329859,0.0,40.635957,...,22.338756,57.523884,3.284246,9.952191,36.076187,13.478254,22.231341,204.279198,29.913855,25.225166


Now this is what our new table looks like


gene_name,TGFb,Retinoic Acid,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,0.00,0,0.377997,0.035485,0.087716,0.263830,0.350411,12.057979,0.319075,53.020671,...,37.376160,182.345536,3.376170,9.657581,36.781075,16.005638,18.488987,72.885622,33.661747,22.205224
1,0.00,50,0.345151,0.034716,0.127103,0.137183,0.000000,13.425112,0.000000,47.729696,...,37.457048,150.343398,2.518407,9.277073,33.313392,16.573176,20.549266,107.455344,33.147009,23.398570
2,0.00,200,0.300917,0.000000,0.162882,0.082244,0.345012,13.311664,0.000000,47.800100,...,30.569839,130.592939,2.773964,7.611645,33.531139,15.295295,20.328332,118.413674,32.668651,22.805652
3,0.00,400,0.203307,0.000000,0.211807,0.069556,0.335723,13.785032,0.000000,46.066403,...,32.521534,123.678421,2.668080,8.206404,32.979449,15.033746,20.089239,121.144959,30.442728,23.403616
4,1.25,0,0.256296,0.000000,0.080188,0.125608,0.000000,11.649623,0.000000,45.712356,...,27.967913,126.723619,3.286000,11.075037,37.483739,14.895510,17.376096,149.058857,32.772074,23.163784
5,1.25,50,0.351086,0.000000,0.054925,0.075671,0.385303,8.973253,0.000000,41.863051,...,27.021822,74.655933,3.736386,10.136228,37.666414,13.484046,21.128904,194.584197,32.281384,23.780587
6,5.00,0,0.230624,0.035837,0.053812,0.110743,0.336408,9.336744,0.193978,43.406801,...,32.606337,110.171030,3.162182,11.979568,35.755054,14.319992,19.077618,153.719588,32.896717,23.078828
7,5.00,200,0.362810,0.000000,0.054184,0.050074,0.676265,9.562037,0.000000,42.264973,...,25.416376,66.817892,3.253249,10.199674,36.682645,13.111032,21.180285,207.092959,31.524216,24.633575
8,10.00,0,0.350926,0.032057,0.092482,0.130880,0.358890,10.128333,0.000000,43.736571,...,28.349972,100.414759,3.536064,12.396366,36.297142,13.755184,18.309939,161.319258,32.275104,22.633430


Cross validation scores: [  -13.63055596 -5531.83157602 -7760.67145271  -919.6598702 ]
Decision Tree Regressor score is 1.0 



Y true


gene_name,TGFb,Retinoic Acid,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
9,10.0,400,0.419643,0.046901,0.0,0.098149,0.376772,10.329859,0.0,40.635957,...,22.338756,57.523884,3.284246,9.952191,36.076187,13.478254,22.231341,204.279198,29.913855,25.225166


Y predict


,TGFb,Retinoic Acid,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,10,400,0.36281,0.0,0.054184,0.050074,0.676265,9.562037,0.0,42.264973,...,25.416376,66.817892,3.253249,10.199674,36.682645,13.111032,21.180285,207.092959,31.524216,24.633575


In [104]:
from sklearn.ensemble import RandomForestRegressor


treatments = ['TGFb', 'Retinoic Acid']
df_dt = getDF_PLSR_Pivoted()
index = 9

print(f'Lets take out row {index}')
display(df_dt.iloc[[index]])
print('Now this is what our new table looks like')
df_dt_minus_row = df_dt.drop(df_dt.index[[index]])
display(df_dt_minus_row)

genes = df_dt_minus_row[df_dt_minus_row.columns.difference(treatments)]


regr = RandomForestRegressor(max_depth=10, random_state=0)
regr.fit(df_dt_minus_row[treatments], genes)

#"cv" cannot be greater than 4 cus I only have 10 samples
# Uses r2 scoring
scores = cross_val_score(regressor, df_dt_minus_row[treatments], genes, cv=4, scoring='explained_variance')
print(f'Cross validation scores: {scores}')
regressor.fit(df_dt_minus_row[treatments], genes)

print(f'Random Forest Regressor score is {regressor.score(df_dt_minus_row[treatments], genes)} \n\n\n')

print('Y true')
display(df_dt.iloc[[index]])

sample_treatment = [[10, 400]]

print('Y predict')
Y_pred = pd.DataFrame(regressor.predict(sample_treatment), columns=genes.columns)
display(pd.DataFrame(np.array(sample_treatment), columns=treatments).join(Y_pred))





Lets take out row 9


gene_name,TGFb,Retinoic Acid,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
9,10.0,400,0.419643,0.046901,0.0,0.098149,0.376772,10.329859,0.0,40.635957,...,22.338756,57.523884,3.284246,9.952191,36.076187,13.478254,22.231341,204.279198,29.913855,25.225166


Now this is what our new table looks like


gene_name,TGFb,Retinoic Acid,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,0.00,0,0.377997,0.035485,0.087716,0.263830,0.350411,12.057979,0.319075,53.020671,...,37.376160,182.345536,3.376170,9.657581,36.781075,16.005638,18.488987,72.885622,33.661747,22.205224
1,0.00,50,0.345151,0.034716,0.127103,0.137183,0.000000,13.425112,0.000000,47.729696,...,37.457048,150.343398,2.518407,9.277073,33.313392,16.573176,20.549266,107.455344,33.147009,23.398570
2,0.00,200,0.300917,0.000000,0.162882,0.082244,0.345012,13.311664,0.000000,47.800100,...,30.569839,130.592939,2.773964,7.611645,33.531139,15.295295,20.328332,118.413674,32.668651,22.805652
3,0.00,400,0.203307,0.000000,0.211807,0.069556,0.335723,13.785032,0.000000,46.066403,...,32.521534,123.678421,2.668080,8.206404,32.979449,15.033746,20.089239,121.144959,30.442728,23.403616
4,1.25,0,0.256296,0.000000,0.080188,0.125608,0.000000,11.649623,0.000000,45.712356,...,27.967913,126.723619,3.286000,11.075037,37.483739,14.895510,17.376096,149.058857,32.772074,23.163784
5,1.25,50,0.351086,0.000000,0.054925,0.075671,0.385303,8.973253,0.000000,41.863051,...,27.021822,74.655933,3.736386,10.136228,37.666414,13.484046,21.128904,194.584197,32.281384,23.780587
6,5.00,0,0.230624,0.035837,0.053812,0.110743,0.336408,9.336744,0.193978,43.406801,...,32.606337,110.171030,3.162182,11.979568,35.755054,14.319992,19.077618,153.719588,32.896717,23.078828
7,5.00,200,0.362810,0.000000,0.054184,0.050074,0.676265,9.562037,0.000000,42.264973,...,25.416376,66.817892,3.253249,10.199674,36.682645,13.111032,21.180285,207.092959,31.524216,24.633575
8,10.00,0,0.350926,0.032057,0.092482,0.130880,0.358890,10.128333,0.000000,43.736571,...,28.349972,100.414759,3.536064,12.396366,36.297142,13.755184,18.309939,161.319258,32.275104,22.633430


Cross validation scores: [  -13.63055596 -5531.83157602 -7760.67145271  -919.6598702 ]
Random Forest Regressor score is 1.0 



Y true


gene_name,TGFb,Retinoic Acid,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
9,10.0,400,0.419643,0.046901,0.0,0.098149,0.376772,10.329859,0.0,40.635957,...,22.338756,57.523884,3.284246,9.952191,36.076187,13.478254,22.231341,204.279198,29.913855,25.225166


Y predict


,TGFb,Retinoic Acid,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,10,400,0.36281,0.0,0.054184,0.050074,0.676265,9.562037,0.0,42.264973,...,25.416376,66.817892,3.253249,10.199674,36.682645,13.111032,21.180285,207.092959,31.524216,24.633575
